### Imports

In [78]:
import numpy as np
from random import randint
from statistics import median
from pandas import read_csv
from PIL import Image, ImageDraw
from statistics import mean
import random

### Clause 1 - KNN 

In [79]:
def euclidean_distance(point1, point2):
    point1_ = np.array(point1)
    point2_ = np.array(point2)

    # calculating Euclidean distance
    # using linalg.norm()
    dist = np.linalg.norm(point1_ - point2_)

    # return Euclidean distance
    return dist


def knn_sort_func(list_):
    return list_['distance']


class KNN:
    def __init__(self, k):
        self.internal_list = []
        self.k = k

    def fit(self, train_data):
        self.internal_list = train_data

    def k_neighbors(self, new_points):
        neighbors_list = []
        for point in new_points:
            neighbors_list.append(self.point_k_neighbors(point[4:132], point[0:4]))
        return neighbors_list

    def point_k_neighbors(self, new_point_f, new_point_n):
        point_k_neighbors_list = []
        for point in self.internal_list:
            point_name = point.get('point_name')
            point_features = point.get('point_data')

            dis = euclidean_distance(point_features, new_point_f)
            point_name_dis = {'point_name': new_point_n, 'NN_name': point_name, 'distance': dis}
            point_k_neighbors_list.append(point_name_dis)
        point_k_neighbors_list.sort(key=knn_sort_func)
        return point_k_neighbors_list[:self.k]

### Clause 2 - ANN _ KD Tree

In [80]:
def get_median(data, dim):
    data_dim = []
    for point in data:
        data_dim.append(point.get("point_data")[dim])
    return median(data_dim)


def get_dim_larger_data(data, dim, med):
    right_data = []
    for point in data:
        if point.get("point_data")[dim] >= med:
            right_data.append(point)
    return right_data


def get_dim_smaller_data(data, dim, med):
    right_data = []
    for point in data:
        if point.get("point_data")[dim] < med:
            right_data.append(point)
    return right_data


def get_neighbors(current, point):
    if len(point) == current.point_dim:
        if len(current.data) > current.leave_size:
            if point[current.current_dim % current.point_dim] >= current.current_median:
                return get_neighbors(current.right, point)
            else:
                return get_neighbors(current.left, point)
        else:
            return current.data
    else:
        return


class OurKdTree:
    def __init__(self, data, leave_size, point_dim, current_dim):
        self.current_median = 0
        self.current_dim = current_dim
        self.point_dim = point_dim
        self.leave_size = leave_size
        self.left = None
        self.right = None
        self.data = data

    def build_tree(self):
        dim_ = self.current_dim % self.point_dim
        if len(self.data) > self.leave_size:
            median_ = get_median(self.data, dim_)
            self.current_median = median_
            right_data = get_dim_larger_data(self.data, dim_, median_)
            self.right = OurKdTree(right_data, self.leave_size, self.point_dim, self.current_dim + 1)

            left_data = get_dim_smaller_data(self.data, dim_, median_)
            self.left = OurKdTree(left_data, self.leave_size, self.point_dim, self.current_dim + 1)

            if self.leave_size <= len(right_data):
                self.right.build_tree()
            if self.leave_size <= len(left_data):
                self.left.build_tree()
        else:
            return self


def get_data(file):
    f_points = []
    data_from_file = read_csv(file)
    points_from_file = data_from_file.values.tolist()
    for point in points_from_file:
        f_points.append({'point_name': point[:4], 'point_data': point[4:]})
    return f_points


def get_point_dim(data):
    if len(data) != 0:
        return len(data[0].get('point_data'))
    return 0


def my_sort_func(e):
    return e['distance']


def remove_dublicates(list_to):
    for i in range(len(list_to) - 1, 0, -1):
        if list_to[i] == list_to[i - 1]:
            del list_to[i]

    return list_to


def flat_candidates_list(candidates):
    flat_list = []
    for candidate in candidates:
        for item in candidate:
            flat_list.append(item)
    return flat_list


def get_final_k_neighbors(candidates, point, k):
    out = flat_candidates_list(candidates)
    res = []
    for candidate in out:
        res.append({'candidate_point': candidate.get('point_name'),
                    'distance': euclidean_distance(candidate.get('point_data'), point)})
    res.sort(key=my_sort_func)
    sorted_list = remove_dublicates(res)
    return sorted_list[0:k]


class ANN:
    def __init__(self, N, L, k):
        self.N = N
        self.L = L
        self.k = k
        self.forest = []
        self.data = None

    def fit(self, train_data):
        self.data = train_data
        for tree in range(self.L):
            self.forest.append(OurKdTree(self.data, self.N, get_point_dim(self.data), tree))
        for tree in self.forest:
            tree.build_tree()

    def k_neighbors(self, new_points):
        neighbors_list = []
        for point in new_points:
            neighbors_list.append({'the_point': point[:4], 'k_neighbors': self.point_k_neighbors(point[4:])})
        return neighbors_list

    def point_k_neighbors(self, new_point):
        candidates = []
        for tree in self.forest:
            candidates.append(get_neighbors(tree, new_point))
        return get_final_k_neighbors(candidates, new_point, self.k)

### Clause 3

In [81]:
def ratio_sort_func(e):
    return e['ratio']


def ratio_method(data):
    relevant_points = []
    for point in data:
        ratio = point.get('k_neighbors')[0].get('distance') / point.get('k_neighbors')[1].get('distance')
        if ratio < 0.8:
            relevant_points.append({'the_point': point, 'ratio': ratio})
    relevant_points.sort(key=ratio_sort_func)
    return relevant_points
    # From 913 point in "Hananya2" data, only 68 point have a nearest neighbor
    # in accordance with "Ratio Test".
    # This method creates a new list that contains only these point
    # It prints the relevant point list, and the length of it (amout of points)


def show_10_points(img1, img2, data):
    im_1 = Image.open(img1)
    im_2 = Image.open(img2)
    for point in data:
        color = '#%06X' % randint(0, 0xFFFFFF)
        draw2 = ImageDraw.Draw(im_2)
        x_point2 = point.get('the_point').get('the_point')[1]
        y_point2 = point.get('the_point').get('the_point')[0]
        draw2.ellipse(((x_point2, y_point2), (x_point2 + 8, y_point2 + 8)), fill=color, outline=color)
        draw1 = ImageDraw.Draw(im_1)
        x_point1 = point.get('the_point').get('the_point')[1]
        y_point1 = point.get('the_point').get('the_point')[0]
        draw1.ellipse(((x_point1, y_point1), (x_point1 + 8, y_point1 + 8)), fill=color, outline=color)
    im_2.show()
    im_1.show()

### Clause 4

In [82]:
def get_ann_result(result, ann_results):
    for ann_result in ann_results:
        if ann_result.get('the_point') == result:
            return ann_result.get('k_neighbors')[0].get('distance')
    return 0


def calculate_error_rate(knn_results, ann_results):
    rate_sum = 0
    for result in knn_results:
        rate_sum = (get_ann_result(result[0].get('point_name'), ann_results) / result[0].get('distance')) + rate_sum
    return ((1 / len(knn_results)) * rate_sum) - 1


def calculate_accuracy(N, L, train, test):
    knn_obj = KNN(1)

    knn_obj.fit(train)
    knn_results = knn_obj.k_neighbors(test)

    ann_obj = ANN(N, L, 1)
    ann_obj.fit(train)
    ann_results = ann_obj.k_neighbors(test)

    error = calculate_error_rate(knn_results, ann_results)
    return error


def cross_validation(N, L, k, train):
    bulk_size = round(len(train) / k)
    bulk_error_rate = []
    for i in range(0, k):
        bulk_train = train[0: (bulk_size * i)] + train[(bulk_size * i) + bulk_size:]
        bulk_test_dic = train[0 + (bulk_size * i):(bulk_size * i) + bulk_size]
        bulk_test = []
        for value in bulk_test_dic:
            temp = value.get('point_name') + value.get('point_data')
            bulk_test.append(temp)
        bulk_error_rate.append(calculate_accuracy(N, L, bulk_train, bulk_test))
    return mean(bulk_error_rate)


def grid_search(k, train):
    n_random_list = []
    l_random_list = []
    # Generate random parameters for L,N
    for i in range(0, 10):
        n = random.randint(2, 30)
        n_random_list.append(n)
        n = random.randint(2, 30)
        l_random_list.append(n)
    print('N 10 values (randomly chosen): ', n_random_list)
    print('L 10 values (randomly chosen): ', l_random_list)

    # Find the lowest error rate using cross_validation() method
    lowest_error = {'error_rate': 1, 'N_value': 0, 'L_value': 0}
    for n in n_random_list:
        for l in l_random_list:
            value = cross_validation(n, l, k, train)
            if value < lowest_error.get('error_rate'):
                lowest_error = {'error_rate': value, 'N_value': n, 'L_value': l}
    print('Optimal combination is : ', lowest_error)
    return lowest_error



### Clause 5

In [83]:
# method to calculate the error of given test data and training data
def error(train_file,test_file,N,L):
    
    # run same training data and test data on both KNN, ANN
    knn = KNN(5)
    knn.fit(train_file)
    
    ann = ANN(N, L, 5)
    ann.fit(train_file)
    test_data = read_csv(test_file)
    test_list = test_data.iloc[0:1, 0:132]
    test_points = test_list.values.tolist()
    
    knn_res = knn.k_neighbors(test_points)
    ann_res = ann.k_neighbors_(test_points)
    
    # calculate sum of distances from KNN
    sum_knn_res = 0
    for list_knn in knn_res:
        for dis_knn in list_knn:
            sum_knn_res += dis_knn
    
    # calculate sum of distances from ANN
    sum_ann_res = 0
    for list_ann in ann_res:
        for dis_ann in list_ann:
            sum_ann_res += dis_ann
            
    # calculate the error   
    return (sum_ann_res/sum_knn_res)-1

# method to calculate the speed of ANN with given files, params
def calc_speed_ann(train_file,test_file,N,L,K):
    t0 = time.process_time()
    ann = ANN(N, L, K)
    ann.fit(train_file)
    test_data = read_csv(test_file)
    test_list = test_data.iloc[0:1, 0:132]
    test_points = test_list.values.tolist()
    ann.k_neighbors_(test_points)
    t1 = time.process_time()
    return t1-t0

def my_func_(list_):
    return list_['speed']


# clause 5 method : returns 5 pairs (L,N) of ANN params that ANN runs fastest, with error =< error given
def five_fastest_param (train_file,test_file):
    fastest_params_andspeed = [] # like {params: [L,N], speed: s}
    for i in range (100):
        N = random.randint(1,10)
        L = random.randint(1,10)
        
        # check error condition:
        if error(train_file,test_file,N,L) < 0.1:
             #  run ANN with L,N (k=3), save params and speed
             params_andspeed = {'params': [N,L], 'speed': calc_speed_ann(train_file,test_file,N,L,3)}
             fastest_params_andspeed.append(params_andspeed)

    # sort by speed
    fastest_params_andspeed.sort(key=my_func_)
    # return 5 fastest
    return fastest_params_andspeed[0:5]

# clause 5 - final (+ plt)
def five_fastest_and_plt (train_file,test_file):
    
    # print five fastest params for given data
    fastest = five_fastest_param (train_file,test_file)
    
    # graph plotting
    params = [] # X = labels
    times = [] # Y = times
    for i in fastest:
        params.append(i['params'])
        times.append(i['speed'])
        
    y_pos = np.arange(len(params))
    plt.bar(y_pos, times, align='center',alpha=0.5)
    plt.xticks(y_pos,params)
    plt.ylabel('Time')
    plt.xlabel('Params')
    plt.title('5 fastest params in ANN, and time taken')
    plt.show
    
    print(fastest)

### Clause 6 (fastest params: N=L=1)

In [84]:
def fastest_params_plt(train_file,test_file):
    
    errors = []
    times = []
    
    for i in range (10):
        p = ANN(1, 1, 5)
        p.fit(train_file)
        data1 = read_csv(test_file)
        the_list1 = data1.iloc[0:1, 0:132]
        points = the_list1.values.tolist()
        r = p.k_neighbors(points)
        print ("Iteration: " + str(i+1))
        print(*r, sep="\n")
        print ("Error: " + str(error(train_file,test_file,1,1)))
        print ("Time: " + str(calc_speed_ann(train_file,test_file,1,1,3))+"\n")
        
        errors.append(error(train_file,test_file,1,1))
        times.append(calc_speed_ann(train_file,test_file,1,1,5))
    
    plt.plot(errors,times)
    plt.ylabel('Time')
    plt.xlabel('Error')
    plt.show

### Clause 7

# Main

In [85]:
train_data_ = get_data("Hananya1.csv")
nn_data = read_csv("Hananya2.csv")
data_list = nn_data.iloc[0:, 0:132]
points_to_test = data_list.values.tolist()

'''
The constructor ANN accepts 3 arguments
1-  N: the maximum number of points existed in each leaf in the tree
2-  L: the number of trees created in our forest
3-  k: number of neighbors we need to get for every point 
'''
# Clause 3 - main
p = ANN(10, 20, 2)
p.fit(train_data_)
r = p.k_neighbors(points_to_test)
best_points = ratio_method(r)
show_10_points("Hananya1.JPG", "Hananya2.JPG", best_points[:10])

# Clause 4 - main
grid_search(4, train_data_)

N 10 values (randomly chosen):  [10, 28, 19, 7, 22, 16, 2, 3, 29, 16]
L 10 values (randomly chosen):  [14, 3, 13, 16, 30, 16, 5, 15, 19, 26]
Optimal combination is :  {'error_rate': 0.006106612987755056, 'N_value': 29, 'L_value': 30}


{'error_rate': 0.006106612987755056, 'N_value': 29, 'L_value': 30}